In [ ]:
!pip install google-genai pydantic

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai
from pydantic import BaseModel, Field

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()

# Set the model for our agents
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Structured Schema for the Evaluator ---
class QualityResult(BaseModel):
    meets_requirements: bool = Field(description="True if the text has excellent flow and tone. False otherwise.")
    feedback: str = Field(description="Detailed critique on what is wrong with the flow and tone. Empty if approved.")

# --- 2. Define the Specialized Agents ---

def generator_agent(current_prompt: str) -> str:
    """The Generator Agent acts on the (potentially updated) prompt to create the output."""
    print("-> [Generator] Drafting content based on current prompt...")

    response = client.models.generate_content(model=MODEL_ID, contents=current_prompt)
    return response.text

def quality_evaluator_agent(content: str) -> QualityResult:
    """The Evaluator Subagent reviews the draft against predefined quality standards."""
    print("-> [Quality Evaluator] Checking flow and tone...")

    prompt = f"""Evaluate the following blog post draft for excellent flow, engaging tone, and clarity.

    Draft:
    {content}

    If it sounds polished, professional, and highly engaging, set meets_requirements to true.
    If it is clunky, boring, or lacks flow, set meets_requirements to false and provide specific feedback on what to fix."""

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': QualityResult,
            'temperature': 0.1, # Keep temperature low for consistent grading
        },
    )
    return response.parsed

def prompt_enhancer_agent(original_prompt: str, evaluation_feedback: str) -> str:
    """The Prompt Enhancer Subagent rewrites the instructions for the Generator based on feedback."""
    print("-> [Prompt Enhancer] Rewriting the prompt to fix issues...")

    prompt = f"""You are an expert prompt engineer. Your job is to update an original prompt
    so that the AI generating the content will fix the issues identified by the evaluator.

    Original Prompt: {original_prompt}
    Evaluator Feedback: {evaluation_feedback}

    Write a NEW, comprehensive prompt that incorporates the original instructions BUT adds strict,
    clear directives to address the evaluator's feedback. Return ONLY the new prompt text."""

    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

# --- 3. Define the Orchestrator (Iterative Loop) ---

def iterative_refinement_workflow(initial_prompt: str, max_iterations: int = 3) -> str:
    """
    The orchestrator manages the iterative refinement loop.
    It progressively improves the output over multiple cycles.
    """
    print("Starting Iterative Refinement Workflow...\n")

    iteration = 1
    current_prompt = initial_prompt
    current_draft = ""

    while iteration <= max_iterations:
        print(f"--- Iteration {iteration} ---")

        # 1. Generator creates output from the prompt
        current_draft = generator_agent(current_prompt)
        print(f"Draft Snippet: {current_draft[:100]}...\n")

        # 2. Evaluator checks the output
        evaluation = quality_evaluator_agent(current_draft)

        # 3. Check exit condition
        if evaluation.meets_requirements:
            print("-> ✅ Evaluator approves! Sending response to user.\n")
            return current_draft
        else:
            print(f"-> ❌ Quality score doesn't meet requirements. Feedback: {evaluation.feedback}\n")

            # 4. Prompt Enhancer creates a better prompt for the next cycle
            current_prompt = prompt_enhancer_agent(current_prompt, evaluation.feedback)
            iteration += 1

    print("-> ⚠️ Max iterations reached to prevent infinite loops. Exiting with latest draft.\n")
    return current_draft

# --- Execution ---
if __name__ == "__main__":
    # Example creative writing task
    task = "Write a short, engaging blog post introducing our new eco-friendly water bottle."

    # Trigger the workflow
    final_output = iterative_refinement_workflow(task)

    print("=== Final Polished Output ===")
    print(final_output)

Starting Iterative Refinement Workflow...

--- Iteration 1 ---
-> [Generator] Drafting content based on current prompt...
Draft Snippet: ## Hydrate Smarter, Live Greener: Introducing the [Your Bottle Name]!

Tired of endlessly buying pla...

-> [Quality Evaluator] Checking flow and tone...
-> ✅ Evaluator approves! Sending response to user.

=== Final Polished Output ===
## Hydrate Smarter, Live Greener: Introducing the [Your Bottle Name]!

Tired of endlessly buying plastic water bottles that clutter our planet and your conscience? We get it. That's why we're absolutely thrilled to unveil our latest innovation: **the [Your Bottle Name]**, designed to revolutionize your hydration routine and make a real difference for the Earth!

Crafted from premium, sustainably sourced materials, the [Your Bottle Name] isn't just another bottle – it's a commitment. We've poured our passion for the planet into every detail, ensuring you get a product that's not only incredibly durable and stylish but al